# **(ADD THE NOTEBOOK NAME HERE)**

## Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write down which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\organ\\Desktop\\Code institute\\FINALPROYECTCODE\\SDG7_Proyect\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\organ\\Desktop\\Code institute\\FINALPROYECTCODE\\SDG7_Proyect'

**Import Libraries**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


**Load Dataset and test it**

In [5]:
raw_path = "Data\Raw\global-data-on-sustainable-energy.csv"
df = pd.read_csv(raw_path, encoding='latin1')
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\organ\AppData\Local\Temp\ipykernel_14716\1918513345.py:1: SyntaxWarning: invalid escape sequence '\R'
  raw_path = "Data\Raw\global-data-on-sustainable-energy.csv"


,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.93911,67.709953


***ETL***

**Rename the colums for easier access**

In [6]:
df.rename(columns={
    'Entity': 'country',
    'Access to electricity (% of population)': 'access_to_electricity',
    'Access to clean fuels for cooking': 'access_to_clean_fuels',
    'Renewable-electricity-generating-capacity-per-capita': 'renewable_capacity_per_capita',
    'Financial flows to developing countries (US $)': 'financial_flows',
    'Renewable energy share in the total final energy consumption (%)': 'renewable_energy_share',
    'Electricity from fossil fuels (TWh)': 'fossil_electricity',
    'Electricity from nuclear (TWh)': 'nuclear_electricity',
    'Electricity from renewables (TWh)': 'renewable_electricity',
    'Low-carbon electricity (% electricity)': 'low_carbon_electricity_pct',
    'Primary energy consumption per capita (kWh/person)': 'primary_energy_per_capita',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 'energy_intensity',
    'Value_co2_emissions_kt_by_country': 'co2_emissions_kt',
    'density\\np/km2': 'density'
}, inplace=True)
# Cleaning column names
df.columns = df.columns.str.strip().str.lower().str.replace('\n', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.rename(columns={
    'density\\np/km2': 'density'
}, inplace=True)
df.head()

,country,year,access_to_electricity,access_to_clean_fuels,renewable_capacity_per_capita,financial_flows,renewable_energy_share,fossil_electricity,nuclear_electricity,renewable_electricity,...,primary_energy_per_capita,energy_intensity,co2_emissions_kt,renewables_%_equivalent_primary_energy,gdp_growth,gdp_per_capita,density,land_areakm2,latitude,longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.93911,67.709953


**Check types**

In [7]:
df.dtypes

country                                    object
year                                        int64
access_to_electricity                     float64
access_to_clean_fuels                     float64
renewable_capacity_per_capita             float64
financial_flows                           float64
renewable_energy_share                    float64
fossil_electricity                        float64
nuclear_electricity                       float64
renewable_electricity                     float64
low_carbon_electricity_pct                float64
primary_energy_per_capita                 float64
energy_intensity                          float64
co2_emissions_kt                          float64
renewables_%_equivalent_primary_energy    float64
gdp_growth                                float64
gdp_per_capita                            float64
density                                    object
land_areakm2                              float64
latitude                                  float64


**Check Null values**

In [8]:
null_columns = df.isnull().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)


access_to_electricity                       10
access_to_clean_fuels                      169
renewable_capacity_per_capita              931
financial_flows                           2089
renewable_energy_share                     194
fossil_electricity                          21
nuclear_electricity                        126
renewable_electricity                       21
low_carbon_electricity_pct                  42
energy_intensity                           207
co2_emissions_kt                           428
renewables_%_equivalent_primary_energy    2137
gdp_growth                                 317
gdp_per_capita                             282
density                                      1
land_areakm2                                 1
latitude                                     1
longitude                                    1
dtype: int64


**Eliminate columns with too many null values and redundant data**

In [9]:
columns_to_drop = ['density', 'land_areakm2', 'latitude', 'longitude', 'renewables_%_equivalent_primary_energy', 'financial_flows']
df.drop(columns=columns_to_drop, inplace=True)
print("Dropped columns:", columns_to_drop)

Dropped columns: ['density', 'land_areakm2', 'latitude', 'longitude', 'renewables_%_equivalent_primary_energy', 'financial_flows']


In [10]:
null_columns = df.isnull().sum()
print(null_columns)

country                            0
year                               0
access_to_electricity             10
access_to_clean_fuels            169
renewable_capacity_per_capita    931
renewable_energy_share           194
fossil_electricity                21
nuclear_electricity              126
renewable_electricity             21
low_carbon_electricity_pct        42
primary_energy_per_capita          0
energy_intensity                 207
co2_emissions_kt                 428
gdp_growth                       317
gdp_per_capita                   282
dtype: int64


**Use interpolate to fill the null values**

In [11]:
df.interpolate(method='linear', inplace=True)
df = df.sort_values(by=['country', 'year'])
df = df.groupby('country').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
null_columns = df.isnull().sum()
print(null_columns)

country                          0
year                             0
access_to_electricity            0
access_to_clean_fuels            0
renewable_capacity_per_capita    0
renewable_energy_share           0
fossil_electricity               0
nuclear_electricity              0
renewable_electricity            0
low_carbon_electricity_pct       0
primary_energy_per_capita        0
energy_intensity                 0
co2_emissions_kt                 0
gdp_growth                       0
gdp_per_capita                   0
dtype: int64


C:\Users\organ\AppData\Local\Temp\ipykernel_14716\1778600730.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
C:\Users\organ\AppData\Local\Temp\ipykernel_14716\1778600730.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.groupby('country').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
C:\Users\organ\AppData\Local\Temp\ipykernel_14716\1778600730.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.groupby('country').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
C:\Users\organ\AppData\

**Save the dataset as a clean csv**

In [13]:
processed_path = "Data/Processed/global-data-on-sustainable-energy-processed.csv"
df.to_csv(processed_path, index=False)
print("Processed data saved to:", processed_path)

Processed data saved to: Data/Processed/global-data-on-sustainable-energy-processed.csv


# Section 1

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [12]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (553063055.py, line 5)